In [20]:
import polars as pl
import json
from web3 import Web3
from eth_abi import decode as decode_abi
from eth_account import Account

mainnet = Web3(Web3.HTTPProvider("https://eth.llamarpc.com"))
anvil = Web3(Web3.HTTPProvider("http://localhost:8545"))

Account.enable_unaudited_hdwallet_features()
account = anvil.eth.account.from_mnemonic(
    "test test test test test test test test test test test junk"
)

In [21]:
df = pl.read_parquet("../../sample.parquet")

print(df.head())
print(df.columns)

shape: (5, 10)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬────────┬───────────┐
│ block_numb ┆ transactio ┆ log_index ┆ transacti ┆ … ┆ topic1    ┆ topic2    ┆ topic3 ┆ data      │
│ er         ┆ n_index    ┆ ---       ┆ on_hash   ┆   ┆ ---       ┆ ---       ┆ ---    ┆ ---       │
│ ---        ┆ ---        ┆ u32       ┆ ---       ┆   ┆ binary    ┆ binary    ┆ binary ┆ binary    │
│ u32        ┆ u32        ┆           ┆ binary    ┆   ┆           ┆           ┆        ┆           │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪════════╪═══════════╡
│ 17780001   ┆ 117        ┆ 411       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]     ┆        ┆ data]     │
│ 17780006   ┆ 109        ┆ 311       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]   

In [22]:
!cast sig-event "Mint(address sender, address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Burn(address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Swap(address indexed sender, address indexed recipient, int256 amount0, int256 amount1, uint160 sqrtPriceX96, uint128 liquidity, int24 tick)"

0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde
0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67


In [23]:
mint_event_bytes = Web3.to_bytes(0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde)
burn_event_bytes = Web3.to_bytes(0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c)
swap_event_bytes = Web3.to_bytes(0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67)

event_df = df.filter((pl.col("topic0") == mint_event_bytes) | (pl.col("topic0") == burn_event_bytes) | (pl.col("topic0") == swap_event_bytes))
print("All events: ", event_df.shape)
print("Mint events: ", event_df.filter(pl.col("topic0") == mint_event_bytes).shape)
print("Burn events: ", event_df.filter(pl.col("topic0") == burn_event_bytes).shape)
print("Swap events: ", event_df.filter(pl.col("topic0") == swap_event_bytes).shape)
event_df.head(8)

All events:  (246, 10)
Mint events:  (4, 10)
Burn events:  (4, 10)
Swap events:  (238, 10)


block_number,transaction_index,log_index,transaction_hash,contract_address,topic0,topic1,topic2,topic3,data
u32,u32,u32,binary,binary,binary,binary,binary,binary,binary
17780001,117,411,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780006,109,311,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780008,13,83,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780009,71,171,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780018,65,203,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,1,14,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,2,21,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780022,157,361,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]


In [24]:
with open("../../broadcast/Anvil.s.sol/31337/run-latest.json", "r") as f:
    deployment = json.load(f)

contract_creates = list(filter(lambda txn: txn['transactionType'] == 'CREATE', deployment['transactions']))

contract_names = [
    'MockERC20',
    'PoolManager',
    'PoolModifyPositionTest',
    'PoolSwapTest',
    'PoolDonateTest',
]

contracts = {}
for contract_name in contract_names:
    with open(f"../../out/{contract_name}.sol/{contract_name}.json", "r") as f:
        abi = json.load(f)['abi']
    
    if contract_name == 'MockERC20':
        # WETH
        address = list(
            filter(
                lambda txn: txn['contractName'] == contract_name and txn['arguments'][0] == 'Mock WETH',
                contract_creates
            )
        )[0]['contractAddress']
        print(f"Mock WETH at {address}")
        contracts['MWETH'] = anvil.eth.contract(address=address, abi=abi)

        # USDC
        address = list(
            filter(
                lambda txn: txn['contractName'] == contract_name and txn['arguments'][0] == 'Mock USDC',
                contract_creates
            )
        )[0]['contractAddress']
        print(f"Mock USDC at {address}")
        contracts['MUSDC'] = anvil.eth.contract(address=address, abi=abi)
    else:
        address = list(filter(lambda txn: txn['contractName'] == contract_name, contract_creates))[0]['contractAddress']
        print(f"Contract {contract_name} at {address}")
        contracts[contract_name] = anvil.eth.contract(address=address, abi=abi)


Mock WETH at 0x5FC8d32690cc91D4c39d9d3abcBD16989F875707
Mock USDC at 0x0165878A594ca255338adfa4d48449f69242Eb8F
Contract PoolManager at 0x5FbDB2315678afecb367f032d93F642f64180aa3
Contract PoolModifyPositionTest at 0x9fE46736679d2D9a65F0992F2272dE9f3c7fa6e0
Contract PoolSwapTest at 0xCf7Ed3AccA5a467e9e704C703E8D87F634fB0Fc9
Contract PoolDonateTest at 0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9


In [25]:
key0 = {
    "currency0": contracts['MWETH'].address,
    "currency1": contracts['MUSDC'].address,
    "fee": 3000,
    "tickSpacing": 60,
    "hooks": "0x0000000000000000000000000000000000000000"
}

key1 = {
    "currency0": contracts['MWETH'].address,
    "currency1": contracts['MUSDC'].address,
    "fee": 3000,
    "tickSpacing": 60,
    "hooks": "0x0c00000000000000000000000000000000000001"
}

In [31]:
contracts['PoolManager'].functions['initialize'](
    key0, 79228162514264337593543950336
).transact({
    'from': account.address,
    'gas': 2_000_000,
    'maxFeePerGas': anvil.to_wei(10, 'gwei'),
    'maxPriorityFeePerGas': anvil.to_wei(2, 'gwei'),
})

contracts['PoolManager'].functions['initialize'](
    key1, 79228162514264337593543950336
).transact({
    'from': account.address,
    'gas': 2_000_000,
    'maxFeePerGas': anvil.to_wei(10, 'gwei'),
    'maxPriorityFeePerGas': anvil.to_wei(2, 'gwei'),
})

HexBytes('0xb405b35d285f5c8575d0883268b93ee5ba881ec16f32d9d2efaa0628070c72ca')

In [26]:
def modifyLiquidity(poolKey, tickLower, tickUpper, liquidityDelta):
    positionParams = {
        "tickLower": tickLower,
        "tickUpper": tickUpper,
        "liquidityDelta": liquidityDelta,
    }
    contracts['PoolModifyPositionTest'].functions["modifyPosition"](
        poolKey,
        positionParams
    )
    
def swap(poolKey, zeroForOne, amountSpecified):
    settings = {
        "withdrawTokens": True,
        "settleUsingTransfer": True
    }
    MIN_PRICE_LIMIT = 4295128739 + 1
    MAX_PRICE_LIMIT = 1461446703485210103287273052203988822378723970342 - 1
    params = {
        "zeroForOne": zeroForOne,
        "amountSpecified": amountSpecified,
        "sqrtPriceLimitX96": MIN_PRICE_LIMIT if zeroForOne else MAX_PRICE_LIMIT,
    }
    contracts['PoolSwapTest'].functions['swap'](
        poolKey,
        params,
        settings
    ).transact({
        'from': account.address,
        'gas': 2_000_000,
        'maxFeePerGas': anvil.to_wei(10, 'gwei'),
        'maxPriorityFeePerGas': anvil.to_wei(2, 'gwei'),
    })


In [27]:
mint_decode_types = ["address", "uint128", "uint256", "uint256"]
burn_decode_types = ["uint128", "uint256", "uint256"]
swap_decode_types = ["int256", "int256", "uint160", "uint128", "int24"]

# loop over the events and simulate the actions against the pools
for row in event_df.iter_rows(named=True):
    if row['topic0'] == mint_event_bytes:
        (sender, amount, amount0, amount1) = decode_abi(mint_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
        
    elif row['topic0'] == burn_event_bytes:
        (amount, amount0, amount1) = decode_abi(burn_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
    elif row['topic0'] == swap_event_bytes:
        (amount0, amount1, sqrtPriceX96, liquidity, tick) = decode_abi(swap_decode_types, row['data'])
        sender = decode_abi(["address"], row['topic1'])
        recipient = decode_abi(["address"], row['topic2'])
        # when amount1 < 0, the user traded token0 for token1
        zeroForOne = True if amount1 < 0 else False
        amountSpecified = amount0 if zeroForOne else amount1
        # swap(key0, zeroForOne=zeroForOne, amountSpecified=amountSpecified)

22720758853106888049 48787266271 236505331117248595095
840825625318141064047 7452951404428 5734063800166926515971
23990493912059353733 226905671155 155986097746152364942
3127085735352943 4112825565 2354945294593178225
